**Simulation groep 2 - RF detection**

De code simuleert hoe drie meetpunten (“Pis” — denk aan drie Raspberry Pi’s of antennes) een zender (“hotspot”) kunnen lokaliseren op basis van het ontvangen RF-signaal (RSSI).
Met ruis en signaalverzwakking wordt berekend hoe goed (of slecht) de schatting van de positie is.

In [13]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from IPython.display import display

# Wat gebeurt bij def rssi_from_distance:

p_tx_dbm: zendvermogen van de raspberry pi's (in dBm)
n: path-loss exponent (hoe snel het signaal afneemt met afstand, typisch 2–4, standaard kiezen wij voor 2.2 maar dat is in te stellen bij de sliders)
noise_db: standaardafwijking van de meetruis

De formule:
    RSSI = Ptx - 10n log (d) + ruis

geeft de signaalsterkte op afstand d

In [14]:
def rssi_from_distance(d, p_tx_dbm= -30.0, n=2.0, noise_db=0.0):
    d = np.maximum(d, 1e-6)
    mean = p_tx_dbm - 10.0 * n * np.log10(d)
    noise = np.random.normal(0.0, noise_db, size=np.shape(d))
    return mean + noise

# Wat gebeurt bij def distance_from_rssi:
Omgekeerde van de vorige formule:
    d = 10 ^ ((Ptx - RSSI)/10n)
Zo schat je de afstand uit de gemeten RSSI

In [15]:
def distance_from_rssi(rssi_dbm, p_tx_dbm=-30.0, n=2.0):
    return 10.0 ** ((p_tx_dbm - rssi_dbm) / (10.0 * n))

# Wat gebeurt bij def multilateration_linear:
Doel: bereken de (x, y)-positie van de hotspot met 3 of meer referentiepunten.
Formule gebaseerd op:
    (x-xi)^2+(y-yi)^2=di^2
    

In [16]:
def multilateration_linear(positions, distances):
    positions = np.asarray(positions, dtype=float)
    distances = np.asarray(distances, dtype=float)
    N = positions.shape[0]
    if N < 3:
        raise ValueError("Minimaal 3 referenties nodig")
    x1, y1 = positions[0]
    d1 = distances[0]
    A = []
    b = []
    for i in range(1, N):
        xi, yi = positions[i]
        di = distances[i]
        Ai = [2*(xi - x1), 2*(yi - y1)]
        bi = (xi**2 + yi**2 - di**2) - (x1**2 + y1**2 - d1**2)
        A.append(Ai)
        b.append(bi)
    A = np.array(A)
    b = np.array(b)
    sol, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    return sol

# Wat gebeurt bij def simulate_and_plot_once:
voert één simulatie uit van een RF-lokalisatie met drie meetpunten (“Pis”) en een zender (“hotspot”).

Stappen:
    Berekent de echte afstand tussen elke Pi en de hotspot.
    Simuleert RSSI-metingen (ontvangen signaalsterkte) op basis van die afstanden, het zendvermogen en wat ruis.
    Schat de afstand terug uit de gemeten RSSI-waarden.
    Voert een trilateratie uit om te bepalen waar de zender zich waarschijnlijk bevindt.
    Vergelijkt de geschatte positie met de echte positie en berekent de foutafstand.
    Toont de resultaten in een tabel en maakt een grafiek met:
        De drie Pis
        De echte hotspot
        De geschatte positie
        Cirkels die de geschatte afstanden voorstellen
    Print de belangrijkste parameters (zendvermogen, path-loss, ruis) en de fout in meters.


In [17]:
def simulate_and_plot_once(pis, hotspot, p_tx_dbm=-30.0, path_loss_n=2.0, noise_db=2.0, fname=None):
    # Bereken echte afstanden tussen Pis en hotspot
    true_distances = np.linalg.norm(pis - hotspot, axis=1)
    
    # Simuleer gemeten RSSI-waarden (met ruis)
    measured_rssi = rssi_from_distance(true_distances, p_tx_dbm=p_tx_dbm, n=path_loss_n, noise_db=noise_db)
    
    # Schat afstanden uit RSSI
    estimated_distances = distance_from_rssi(measured_rssi, p_tx_dbm=p_tx_dbm, n=path_loss_n)
    
    # Bepaal geschatte positie via trilateratie
    est_pos = multilateration_linear(pis, estimated_distances)
    
    # Bereken fout (afstand tussen echt en geschat punt)
    est_error = np.linalg.norm(est_pos - hotspot)

    # Resultaten in tabelvorm
    df = pd.DataFrame({
        "Pi": ["Pi1","Pi2","Pi3"],
        "x (m)": np.round(pis[:,0],3),
        "y (m)": np.round(pis[:,1],3),
        "true_distance (m)": np.round(true_distances,3),
        "measured_RSSI (dBm)": np.round(measured_rssi,3),
        "estimated_distance (m)": np.round(estimated_distances,3)
    })

    # Toon tabel (werkt in Jupyter of standaard weergave)
    try:
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user("RF measurements", df)
    except Exception:
        display(df)

    # Plot posities en schatting
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_aspect('equal')
    ax.scatter(pis[:,0], pis[:,1], s=80)
    for i, (xx,yy) in enumerate(pis):
        ax.annotate(f"Pi{i+1}\n({xx:.1f},{yy:.1f})", (xx,yy), textcoords="offset points", xytext=(4,4))
    ax.scatter([hotspot[0]], [hotspot[1]], marker='x', s=100)
    ax.annotate(f"Hotspot true\n({hotspot[0]:.1f},{hotspot[1]:.1f})", (hotspot[0],hotspot[1]), textcoords="offset points", xytext=(4,-12))
    ax.scatter([est_pos[0]], [est_pos[1]], marker='o', s=80)
    ax.annotate(f"Est. ({est_pos[0]:.2f},{est_pos[1]:.2f})\n err={est_error:.2f} m", (est_pos[0], est_pos[1]), textcoords="offset points", xytext=(6,6))

    # Cirkels rond Pis op basis van geschatte afstand
    theta = np.linspace(0, 2*np.pi, 200)
    for i in range(3):
        r = estimated_distances[i]
        cx, cy = pis[i]
        ax.plot(cx + r*np.cos(theta), cy + r*np.sin(theta))

    # Opmaak van de plot
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_title("Simulatie: RSSI-based trilateratie (single run)")
    ax.grid(True)

    # Opslaan als bestand (optioneel)
    if fname:
        fig.savefig(fname, dpi=150)
        print(f"Grafiek opgeslagen naar: {fname}")
    plt.show()

    # Print resultaten
    print(f"P_tx = {p_tx_dbm} dBm, n = {path_loss_n}, noise = {noise_db} dB")
    print(f"True position: ({hotspot[0]:.3f}, {hotspot[1]:.3f})")
    print(f"Estimated: ({est_pos[0]:.3f}, {est_pos[1]:.3f}) — fout: {est_error:.3f} m")


In dit deel van de code wordt een interactieve gebruikersinterface gebouwd met behulp van het pakket ipywidgets.
Met deze sliders kun je zelf spelen met de parameters van de RF-simulatie en direct zien hoe dat de positiebepaling beïnvloedt.

Je kunt aanpassen:
de x- en y-coördinaten van drie Pis (ontvangers),
de positie van de hotspot (de zender),
het zendvermogen in dBm (P_tx),
de path-loss exponent (n), die bepaalt hoe snel het signaal verzwakt met afstand,
en de ruis (noise σ), die meetonzekerheid simuleert.

Elke keer dat je een slider beweegt, wordt de functie
simulate_and_plot_once() opnieuw uitgevoerd.

Die functie berekent:
de echte afstanden tussen de Pis en de hotspot,
de gesimuleerde RSSI-metingen,
de geschatte afstanden via het model,
en de berekende positie van de hotspot met behulp van trilateratie.

Daarna wordt meteen een grafiek getoond waarin:
de posities van de Pis,
de echte hotspot,
en de geschatte positie met foutmarge zichtbaar zijn.

In [18]:
try:
    import ipywidgets as widgets
    from ipywidgets import FloatSlider, interact, VBox, HBox, Layout

    # Stel een standaard breedte in voor de sliders
    layout = Layout(width='320px')

    # Sliders voor de coördinaten van de drie Pis
    pi_slider_x1 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi1 x', layout=layout)
    pi_slider_y1 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi1 y', layout=layout)
    pi_slider_x2 = FloatSlider(value=10.0, min=-20.0, max=20.0, step=0.5, description='Pi2 x', layout=layout)
    pi_slider_y2 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi2 y', layout=layout)
    pi_slider_x3 = FloatSlider(value=5.0, min=-20.0, max=20.0, step=0.5, description='Pi3 x', layout=layout)
    pi_slider_y3 = FloatSlider(value=8.0, min=-20.0, max=20.0, step=0.5, description='Pi3 y', layout=layout)

    # Sliders voor de positie van de hotspot
    hot_x = FloatSlider(value=4.0, min=-20.0, max=20.0, step=0.5, description='Hotspot x', layout=layout)
    hot_y = FloatSlider(value=3.0, min=-20.0, max=20.0, step=0.5, description='Hotspot y', layout=layout)

    # Sliders voor zendvermogen, path-loss exponent en ruis
    p_tx_slider = FloatSlider(value=-30.0, min=-90.0, max=0.0, step=1.0, description='P_tx (dBm)', layout=layout)
    n_slider = FloatSlider(value=2.0, min=1.0, max=4.5, step=0.1, description='path-loss n', layout=layout)
    noise_slider = FloatSlider(value=2.0, min=0.0, max=12.0, step=0.1, description='noise σ (dB)', layout=layout)

    # Functie die bij elke wijziging van een slider opnieuw de simulatie uitvoert
    def interactive_sim(x1,y1,x2,y2,x3,y3,hx,hy,p_tx_dbm,path_loss_n,noise_db):
        pis = np.array([[x1,y1],[x2,y2],[x3,y3]])     # Posities van de Pis
        hotspot = np.array([hx,hy])                   # Positie van de hotspot
        simulate_and_plot_once(pis, hotspot, p_tx_dbm=p_tx_dbm, path_loss_n=path_loss_n, noise_db=noise_db)

    # Bouw de lay-out van de interface met twee kolommen: Pi-coördinaten links, parameters rechts
    ui = VBox([
        HBox([
            VBox([pi_slider_x1, pi_slider_y1, pi_slider_x2, pi_slider_y2, pi_slider_x3, pi_slider_y3]),
            VBox([hot_x, hot_y, p_tx_slider, n_slider, noise_slider])
        ])
    ])

    # Koppel de sliders aan de simulatiefunctie
    out = widgets.interactive_output(interactive_sim, {
        'x1': pi_slider_x1, 'y1': pi_slider_y1,
        'x2': pi_slider_x2, 'y2': pi_slider_y2,
        'x3': pi_slider_x3, 'y3': pi_slider_y3,
        'hx': hot_x, 'hy': hot_y,
        'p_tx_dbm': p_tx_slider, 'path_loss_n': n_slider, 'noise_db': noise_slider
    })

    # Toon de sliders en de uitvoer (grafiek)
    display(ui, out)
    print("ipywidgets beschikbaar — interactieve sliders getoond.")

# Als ipywidgets niet werkt (dan moet deze package nog geinstalleerd worden), voer één vaste simulatie uit
except Exception as e:
    print("ipywidgets niet beschikbaar in deze omgeving — voer een single-run voorbeeld uit.")
    
    # Standaardposities voor drie Pis en een hotspot
    pis = np.array([[0.0,0.0],[10.0,0.0],[5.0,8.0]])
    hotspot = np.array([4.0,3.0])
    out_file = "/mnt/data/rf_simulation_example.png"

    # Simuleer één run en sla het resultaat op als afbeelding
    simulate_and_plot_once(pis, hotspot, p_tx_dbm=-30.0, path_loss_n=2.0, noise_db=2.0, fname=out_file)
    print(f"[Download de afbeelding](/mnt/data/rf_simulation_example.png)")


Output()

ipywidgets beschikbaar — interactieve sliders getoond.
